### Import Library

In [1]:
# import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

from scipy.sparse import csr_matrix, vstack

import pickle
from datetime import datetime

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

### Import the feature-extracted data

In [2]:
song_df= pd.read_pickle('../../data_lyrics/pd/fx_th_songs.pkl')

In [3]:
song_df.head()

,song_name,href,lyric,artist,lines,words,n_words,words_str,artists,duplicates,n_artists,n_unique_words,unique_words_ratio,words_per_line,tf_idf_vector,tf_idf_score
0,1 Of 1,/music/thailyric/23579,Look at me now ชีวิตกูสบาย Get high อยู่ข้างใน...,1mill,31,"[look, at, me, now, ชีวิต, กู, สบาย, get, high...",307,look at me now ชีวิต กู สบาย get high อยู่ ข้า...,{1mill},0,1,140,0.456026,9.903226,"(0, 27739)\t0.05469112958757089\n (0, 27719...",8.292645
1,2Turnt,/music/thailyric/23578,Ay can you turn a headphone upIm too turnt rig...,1mill,47,"[ay, can, you, turn, a, headphon, upim, too, t...",370,ay can you turn a headphon upim too turnt righ...,{1mill},0,1,220,0.594595,7.872340,"(0, 27834)\t0.04916141776286795\n (0, 27778...",10.329311
2,Fu*k It Up,/music/thailyric/23565,I just be takin these drugs takin these drugsT...,1mill,121,"[i, just, be, takin, these, drug, takin, these...",817,i just be takin these drug takin these drugsta...,{1mill},0,1,322,0.394125,6.752066,"(0, 27836)\t0.011114755400495453\n (0, 2768...",11.707533
3,On My Own,/music/thailyric/23566,I like counting cash บอกพวกแม่งไม่ต้องโอนมึงไม...,1mill,42,"[i, like, count, cash, บอก, กรุป, มามี้, ง, ไม...",327,i like count cash บอก กรุป มามี้ ง ไม่ต้อง โอน...,{1mill},0,1,135,0.412844,7.785714,"(0, 27895)\t0.11052772529675105\n (0, 27849...",7.643345
4,OnSomeShit!,/music/thailyric/23577,I get the racks I get the bag พวกกูชอบ CashI j...,1mill,97,"[i, get, the, rack, i, get, the, bag, กรุป, กู...",574,i get the rack i get the bag กรุป กู ชอบ cashi...,{1mill},0,1,245,0.426829,5.917526,"(0, 27672)\t0.006885052209383035\n (0, 2762...",10.457253


In [4]:
song_df.dtypes

song_name              object
href                   object
lyric                  object
artist                 object
lines                   int64
words                  object
n_words                 int64
words_str              object
artists                object
duplicates              int64
n_artists               int64
n_unique_words          int64
unique_words_ratio    float64
words_per_line        float64
tf_idf_vector          object
tf_idf_score          float64
dtype: object

## Prediction

In [5]:
# parameter
# number of sets
n_set = {'train': 20, 'val': 20}

# number of artists per set
n_artist = 3

# minimum number of songs of one artist
n_song_min = 5

# maximum number of song - artist pairs per artist set
n_song_artist_max = 100

In [6]:
song_df.columns

Index(['song_name', 'href', 'lyric', 'artist', 'lines', 'words', 'n_words',
       'words_str', 'artists', 'duplicates', 'n_artists', 'n_unique_words',
       'unique_words_ratio', 'words_per_line', 'tf_idf_vector',
       'tf_idf_score'],
      dtype='object')

In [7]:
def select_artist_song_create_feature(song_df, n_set, n_artist, n_song_min, n_song_artist_max):
    song_count_df = song_df.groupby('artist')[['artist']].count().rename(columns={'artist': 'count'})
    artist_lst = list(song_count_df.loc[song_count_df['count'] >= n_song_min].index.values)

    n_set_total = sum(n_set.values())

    artist_set = []
    while len(artist_set) < n_set_total:
        new_artist = tuple(np.random.choice(artist_lst, size=n_artist, replace=False))
        if new_artist not in artist_set:
            artist_set.append(new_artist)

    # split artist sets
    artist_select = {}
    for field, n in n_set.items():
        i_select = np.random.choice(range(len(artist_set)), size=n, replace=False)
        artist_list = list(artist_set)
        artist_select[field] = [artist_list[i] for i in i_select]
        artist_set = [s for s in artist_set if s not in artist_select[field]]
    # create dataframe with all features
    feature_dict = {}
    # dictionary to map artist set id to list of artists
    set_id_to_artist_tp = {}

    i = 0
    for field, artist_set in artist_select.items():
        df_lst = []
        for artist_tp in artist_set:
            i += 1
            df = song_df.loc[song_df['artist'].isin(artist_tp), 
                             ['artist', 'song_name', 'n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_vector', 
                              'tf_idf_score']]
            # check if number of songs is too high
            if len(df) * n_artist > n_song_artist_max:
                df = df.sample(int(n_song_artist_max / n_artist), random_state=0)
                
            df['artist_set_id'] = i
            set_id_to_artist_tp[i] = artist_tp
            df_lst.append(df)
        feature_dict[field] = pd.concat(df_lst)  
        print('Number of songs in {}: {}'.format(field, len(feature_dict[field])))

    # get all selected artists
    artist_select_set = set.union(*[set(sum(tp_lst, ())) for tp_lst in artist_select.values()])

    # calculate mean vector
    def get_mean_vector(vec_lst):
        return csr_matrix(vstack(vec_lst).mean(axis=0))

    # create artist dataframe from training data
    df_lst = []
    for artist, df in song_df.loc[song_df['artist'].isin(artist_select_set)].groupby('artist'):
        dic = {'artist': artist}
        # calculate averages and standard diviations
        for field in ['n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_score']:
            dic[field + '_mean'] = df[field].mean()
            dic[field + '_std'] = df[field].std()

        # number of songs
        dic['songs'] = len(df)

        # calculate average tf idf vector
        dic['tf_idf_vector_mean'] = get_mean_vector(df['tf_idf_vector'])

        df_lst.append(pd.DataFrame(dic, index=[0]))
    artist_feature_df = pd.concat(df_lst)

    def get_features(df):
        # get artist set id
        artist_set_id = df['artist_set_id'].iloc[0]
        
        # get all artists
        artist_feature_select_df = artist_feature_df.loc[artist_feature_df['artist']\
                                                         .isin(set_id_to_artist_tp[artist_set_id])]

        # merge dataframes
        artist_song_feature_df = pd.merge(artist_feature_select_df.assign(key=0), df.assign(key=0), on='key', 
                                          suffixes=['_artist', '_song']).drop('key', axis=1)    
        artist_song_feature_df['same_artist'] = \
            artist_song_feature_df['artist_artist'] == artist_song_feature_df['artist_song']

        # calculate features
        # add feature polarity
        for feature in ['n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_score']:
            artist_song_feature_df[feature + '_diff'] = \
                artist_song_feature_df[feature] - artist_song_feature_df[feature + '_mean']
            artist_song_feature_df[feature + '_diff_std'] = \
                artist_song_feature_df[feature + '_diff'] / artist_song_feature_df[feature + '_std']
        
        # calculate similarity of artist tf idf vector and song vector
        def tf_idf_vector_similarity(artist_vector, song_vector, songs, same_artist):
            # check if song is from same artist
            if same_artist:
                # deduct song vector from artist vector
                artist_vector = (songs * artist_vector - song_vector) / (songs - 1)
            # calculate similarity
            return cosine_similarity(artist_vector, song_vector)[0][0]

        # calculate vector similarity between artist and song
        artist_song_feature_df['vector_similarity'] = \
            artist_song_feature_df.apply(lambda row: tf_idf_vector_similarity(row['tf_idf_vector_mean'], 
                                                      row['tf_idf_vector'], row['songs'], row['same_artist']), axis=1)    
        return artist_song_feature_df

    artist_song_feature = {}
    for field in feature_dict:
        artist_song_feature[field] = feature_dict[field].groupby('artist_set_id').apply(get_features)\
                                                        .reset_index(drop=True)
        
    return artist_song_feature

In [8]:
np.random.seed(0)
artist_song_feature = select_artist_song_create_feature(song_df, n_set, n_artist, n_song_min, n_song_artist_max)

Number of songs in train: 660
Number of songs in val: 660


In [9]:
artist_song_feature['train'].iloc[0]

artist_artist                                                      image_suthita
n_words_mean                                                          179.809524
n_words_std                                                            50.108501
unique_words_ratio_mean                                                 0.404315
unique_words_ratio_std                                                  0.092661
words_per_line_mean                                                     5.886216
words_per_line_std                                                      1.629727
tf_idf_score_mean                                                       6.640067
tf_idf_score_std                                                        1.147122
songs                                                                         21
tf_idf_vector_mean               (0, 3)\t0.006666676917507676\n  (0, 182)\t0....
artist_song                                                               liltan
song_name                   

In [10]:
feature = ['n_words_diff', 'n_words_diff_std',
        'unique_words_ratio_diff', 'unique_words_ratio_diff_std',
        'words_per_line_diff', 'words_per_line_diff_std', 'tf_idf_score_diff',
        'tf_idf_score_diff_std','vector_similarity']
df_lst = []

pd.options.mode.chained_assignment = None  # default='warn'

for f in feature:
    df = artist_song_feature['train'][['same_artist']]
    df['feature'] = f
    df['value'] = artist_song_feature['train'][f]
    df_lst.append(df)
    
feature_df = pd.concat(df_lst)
feature_df.head()

,same_artist,feature,value
0,False,n_words_diff,404.190476
1,True,n_words_diff,13.190476
2,False,n_words_diff,279.190476
3,False,n_words_diff,353.190476
4,False,n_words_diff,149.190476


In [11]:
feature_df.head()

,same_artist,feature,value
0,False,n_words_diff,404.190476
1,True,n_words_diff,13.190476
2,False,n_words_diff,279.190476
3,False,n_words_diff,353.190476
4,False,n_words_diff,149.190476


In [12]:
def violine_feature_plot(feature_df, feature_select):

    fig = go.Figure()
    df = feature_df.loc[feature_df['feature'].isin(feature_select)]

    fig.add_trace(go.Violin(x=df['feature'][df['same_artist']],
                            y=df['value'][df['same_artist']],
                            legendgroup='Same Artist', scalegroup='Same Artist', name='Same Artist',
                            side='negative')
                 )
    fig.add_trace(go.Violin(x=df['feature'][~df['same_artist']],
                            y=df['value'][~df['same_artist']],
                            legendgroup='Different Artists', scalegroup='Different Artists', name='Different Artists',
                            side='positive')
                 )

    fig.update_traces(meanline_visible=True)
    fig.update_layout(violingap=0, violinmode='overlay')
    fig.update_layout(title='Feature Comparison')
    fig.update_xaxes(title='Feature')
    return fig

In [13]:
feature_df

,same_artist,feature,value
0,False,n_words_diff,404.190476
1,True,n_words_diff,13.190476
2,False,n_words_diff,279.190476
3,False,n_words_diff,353.190476
4,False,n_words_diff,149.190476
...,...,...,...
1975,False,vector_similarity,0.379273
1976,True,vector_similarity,0.070689
1977,True,vector_similarity,0.266614
1978,False,vector_similarity,0.417765


In [14]:
fig = violine_feature_plot(feature_df, feature[1:])
fig.update_layout(
    autosize=False,
    width=2000,
    height=800,)
fig.update_xaxes(range=[-0.5, 4.5])
fig.show()

# Note that you should click *Autoscale* on the figure option to show all artists' violins

In [15]:
fig = violine_feature_plot(feature_df, ['n_words_diff_std', 'unique_words_ratio_diff_std', 'words_per_line_diff_std', 'tf_idf_score_diff_std'])
fig.update_layout(
    autosize=False,
    width=1000,
    height=800,)
fig.update_xaxes(range=[-0.5, 4.5])
fig.show()

In [16]:
fig = violine_feature_plot(feature_df, ['vector_similarity'])
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig.update_xaxes(range=[-1, 1])
fig.show()

### Prepare data for prediction

In [17]:
def prepare_data(df, feature_org, feature_abs):
    for f in feature_abs:
        df[f] = df[f].abs()
    X = df[feature_org + feature_abs].values
    y = df['same_artist'].values
    
    return X, y

def select_songs_train_pipeline(song_df, n_set, n_artist, n_song_min, n_song_artist_max, feature_org, feature_abs, pipeline):
    artist_song_feature = select_artist_song_create_feature(song_df, n_set, n_artist, n_song_min, n_song_artist_max)

    # prepare data
    X, y = prepare_data(artist_song_feature['train'], feature_org, feature_abs)

    pipeline = pipeline.fit(X, y)
    
    return artist_song_feature, pipeline

In [18]:
# prepare data create and train pipeline
n_artist = 3
n_song_min = 5
n_set = {'train': 100}
n_song_artist_max = 100

feature_org = ['n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_score', 'vector_similarity']
feature_abs = ['n_words_diff', 'n_words_diff_std', 'unique_words_ratio_diff', 'unique_words_ratio_diff_std', 
               'words_per_line_diff', 'words_per_line_diff_std', 'tf_idf_score_diff', 'tf_idf_score_diff_std']

pipeline = Pipeline([('scale', StandardScaler()), 
                     ('clf', LogisticRegression(solver='lbfgs', max_iter=3000, 
                                                class_weight={False: 1/n_artist, True:(n_artist - 1)/n_artist}))])

np.random.seed(1)
artist_song_feature, pipeline = select_songs_train_pipeline(song_df, n_set, n_artist, n_song_min, n_song_artist_max, 
                                                            feature_org, feature_abs, pipeline)

Number of songs in train: 3296


In [19]:
feature_importance_df = pd.DataFrame({'feature': feature_org+feature_abs, 'coefficient':pipeline['clf'].coef_[0]})

px.bar(feature_importance_df.sort_values('coefficient'), x='feature', y='coefficient')

## Validation

In [20]:
def predict_artist(df, feature_org, feature_abs, pipeline, top_n):
    # prepare data
    X, y = prepare_data(df, feature_org, feature_abs)
    
    # get probability
    proba = pipeline.predict_proba(X)
    # attach to dataframe
    df['probability'] = proba[:, 1]
    df['correct_prediction'] = df['artist_artist'] == df['artist_song']
    
    # get artist song pairs with highest probability
    predict_select = df.sort_values('probability', ascending=False).groupby(['artist_set_id']).head(top_n)\
                       .groupby(['artist_set_id'])['correct_prediction'].max()
    
    # print(predict_select)
    # get accuracy
    print('Accuracy: {}'.format(predict_select.mean()))
    
    return predict_select

In [21]:
artist_predict_df = predict_artist(artist_song_feature['train'], feature_org, feature_abs, pipeline, top_n=1)

Accuracy: 0.8


In [22]:
artist_predict_df = predict_artist(artist_song_feature['train'], feature_org, feature_abs, pipeline, top_n=2)

Accuracy: 0.91


In [23]:
n_artist_lst = [2, 4, 8, 16, 32, 64, 128]
top_n_lst = [1, 2, 4, 8, 16, 32, 64]
n_song_artist_max = 128
np.random.seed(2)

n_set = {'train': 100, 'val': 100}

feature_org = ['n_words', 'unique_words_ratio', 'words_per_line', 'tf_idf_score', 'vector_similarity']
feature_abs = ['n_words_diff', 'n_words_diff_std', 'unique_words_ratio_diff', 'unique_words_ratio_diff_std', 
               'words_per_line_diff', 'words_per_line_diff_std', 'tf_idf_score_diff', 'tf_idf_score_diff_std', ]

pipeline = Pipeline([('scale', StandardScaler()), 
                     ('clf', LogisticRegression(solver='lbfgs', max_iter=3000, 
                                                class_weight={False: 1/n_artist, True:(n_artist - 1)/n_artist}))])

result_lst = []

for n_artist in n_artist_lst:
    print(datetime.now())
    print('n_artist: {}'.format(n_artist))
    
    artist_song_feature, pipeline = select_songs_train_pipeline(song_df, n_set, n_artist, n_song_min, 
                                                                n_song_artist_max, feature_org, feature_abs, pipeline)
    
    for top_n in [n for n in top_n_lst if n < n_artist]:
        print('top_n: {}'.format(top_n))
        
        predict_select = predict_artist(artist_song_feature['val'], feature_org, feature_abs, pipeline, top_n=top_n)
        
        result_dict = {'n_artist': n_artist, 'top_n': top_n, 'accuracy': predict_select.mean()}
        result_lst.append(result_dict)
        
    print('')
    
result_df = pd.DataFrame(result_lst)

2022-05-24 22:45:01.752647
n_artist: 2
Number of songs in train: 4633
Number of songs in val: 4550
top_n: 1
Accuracy: 0.89

2022-05-24 22:45:15.929344
n_artist: 4
Number of songs in train: 3200
Number of songs in val: 3200
top_n: 1
Accuracy: 0.85
top_n: 2
Accuracy: 0.95

2022-05-24 22:45:33.124575
n_artist: 8
Number of songs in train: 1600
Number of songs in val: 1600
top_n: 1
Accuracy: 0.7
top_n: 2
Accuracy: 0.84
top_n: 4
Accuracy: 0.94

2022-05-24 22:45:49.710193
n_artist: 16
Number of songs in train: 800
Number of songs in val: 800
top_n: 1
Accuracy: 0.5
top_n: 2
Accuracy: 0.66
top_n: 4
Accuracy: 0.81
top_n: 8
Accuracy: 0.95

2022-05-24 22:46:06.808886
n_artist: 32
Number of songs in train: 400
Number of songs in val: 400
top_n: 1
Accuracy: 0.31
top_n: 2
Accuracy: 0.51
top_n: 4
Accuracy: 0.64
top_n: 8
Accuracy: 0.73
top_n: 16
Accuracy: 0.92

2022-05-24 22:46:23.297656
n_artist: 64
Number of songs in train: 200
Number of songs in val: 200
top_n: 1
Accuracy: 0.19
top_n: 2
Accuracy: 0.

In [31]:
fig = px.line(result_df, x='n_artist', y='accuracy', color='top_n', 
              title='Accuracy vs number of artist and number of top selections', 
              labels={'n_artist': 'Number of artists per set', 'top_n': 'Top predictions'}).update_traces(mode='lines+markers')
fig.update_layout(
    autosize=False,
    width=1000,
    height=600,)
fig.show()